In [ ]:
fig = plt.figure(figsize=(4.2,2.4))
ax1 = fig.add_subplot(1,2,1) 
plot_2d_prob_density(X, Y, kde_Z, ax=ax1,
                     xlabel='x'+speed_unit_text, ylabel='y'+speed_unit_text, colorbar=False)
ax1.grid(False)
ax2 = fig.add_subplot(1,2,2) 
plot_2d_prob_density(X, Y, pdf_Z, ax=ax2,
                     xlabel='x'+speed_unit_text, ylabel='y'+speed_unit_text, colorbar=False)
ax2.grid(False)
ax2.get_yaxis().set_visible(False)

In [ ]:
%%time
true_R_square_mean_set=[]
for year_length in [5,10]:
    start_year, end_year = df_all_years.index.year[0], 2015-year_length+1
    df_standard = df_all_years[str(2015-year_length+1):str(2015)]
    true_R_square_set = []
    for sub_start_year in arange(start_year, end_year+1):
        sub_end_year = sub_start_year+year_length-1
        df_previous =  df_all_years[str(sub_start_year):str(sub_end_year)]
        density_collection=Parallel(n_jobs=-1)(delayed(direction_compare)(None, df_standard, angle, incre, True, df_previous) 
                                            for angle in arange(0, 360, incre))  
        true_R_square_set.append(true_R_square(density_collection))
    true_R_square_set=pd.DataFrame(true_R_square_set, index=arange(start_year, end_year+1))
    year_lim = end_year-year_length-5, end_year-year_length
    true_R_square_mean = true_R_square_set.query('index >= @year_lim[0] & index <= @year_lim[1]').mean()
    true_R_square_mean['year_lim']=year_lim
    true_R_square_mean_set.append(true_R_square_mean)
    plot(true_R_square_set, label=year_length)
plt_configure(legend=True)
display(pd.DataFrame(true_R_square_mean_set).set_index('year_lim'))